In [1520]:
import numpy as np

### 初次攻击分布

In [1521]:
cd_weight = np.concatenate([np.full(185, 15), np.arange(15, 0, -1)])


### 攻击机制

In [1522]:
atk_offset = np.array([74] + [28] * 3)
atk_offset = np.cumsum(atk_offset)
print(atk_offset)

[ 74 102 130 158]


In [1523]:
# dim 0: 不被偷(0), 被偷(1)
# dim 1: 造成伤害次数(1 - 8)的权重
weight_mtx = np.zeros((2,9), dtype=np.int64)
weight_sum = (cd_weight.sum() * 15).astype(np.int64)
for cd in range(1, 201):
    for intv in range(186, 201):
        hit_times = np.array([cd, intv])
        hit_times = np.cumsum(hit_times)
        hit_times = hit_times[:, np.newaxis] + atk_offset
        hit_times = hit_times.reshape(-1)

        dmg_num_not_stolen = np.searchsorted(hit_times, 363, side='right')
        dmg_num_stolen = np.searchsorted(hit_times, 300, side='right')
        
        weight_mtx[0, dmg_num_not_stolen] += cd_weight[cd - 1]
        weight_mtx[1, dmg_num_stolen] += cd_weight[cd - 1]
        
gcd_val = np.gcd.reduce(weight_mtx.flatten())
weight_mtx //= gcd_val
weight_sum //= gcd_val

In [1524]:
for i in range(2):
    print(f"{'被偷' if i else '不被偷'} 造成伤害次数权重分布")
    for dmg_num in range(1, 9):
        print(f"{dmg_num}次 {weight_mtx[i, dmg_num]/weight_sum:.10f} = {weight_mtx[i, dmg_num]:3d} / {weight_sum:3d}")
    print()


不被偷 造成伤害次数权重分布
1次 0.0000000000 =   0 / 965
2次 0.0000000000 =   0 / 965
3次 0.0000000000 =   0 / 965
4次 0.5025906736 = 485 / 965
5次 0.1450777202 = 140 / 965
6次 0.1450777202 = 140 / 965
7次 0.1450777202 = 140 / 965
8次 0.0621761658 =  60 / 965

被偷 造成伤害次数权重分布
1次 0.0010362694 =   1 / 965
2次 0.1181347150 = 114 / 965
3次 0.1450777202 = 140 / 965
4次 0.5647668394 = 545 / 965
5次 0.1440414508 = 139 / 965
6次 0.0269430052 =  26 / 965
7次 0.0000000000 =   0 / 965
8次 0.0000000000 =   0 / 965



In [1525]:
# n-1 曾不被偷 + 1 曾被偷 造成伤害次数的权重分布
for n in range(3, 7):
    weight_ans = weight_mtx[0].copy()
    for _ in range(n - 2):
        weight_ans = np.convolve(weight_ans, weight_mtx[0])
    weight_ans = np.convolve(weight_ans, weight_mtx[1])

    p, q = weight_ans[:23].sum(), weight_sum ** n
    gcd_val = np.gcd(p, q)
    p, q = p // gcd_val, q // gcd_val
    print(f"{n}曾 {p/q:.10f} = {p:12d} / {q:14d}")

3曾 1.0000000000 =            1 /              1
4曾 0.8997060104 =   6241656469 /     6937440005
5曾 0.2785109064 = 372905471829 /  1338925920965
6曾 0.0038695398 = 199987645779 / 51682540549249


In [1526]:
# 不在宝前方法
import math
a1 = np.array([
    [83 + 2*i for i in range(15)],
    [28] * 15,
    [28] * 15,
    [28] * 15,
    [19 - i for i in range(15)],
], dtype=np.int64).tolist()
a2 = np.array([[186 + i]*5 for i in range(15)]).T.tolist()
weight_sum = math.lcm(*a2[0])
weight_mtx = [
    [
        weight_sum // a2[i][j] * a1[i][j] 
        for j in range(15)
    ] 
    for i in range(5)
]
weight_mtx = [
    sum(row) / weight_sum / 15
    for row in weight_mtx
]
weight_mtx

[0.5018396002379251,
 0.14515048855826332,
 0.14515048855826332,
 0.14515048855826332,
 0.06270893408728495]